In [ ]:
import datetime
import uuid

from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType
from pyspark.sql import SparkSession
from configs import kafka_config
import os

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1 pyspark-shell"
)

spark = (
    SparkSession.builder.appName("KafkaStreaming")
    .master("local[*]")
    .config("spark.sql.debug.maxToStringFields", "200")
    .config("spark.sql.columnNameLengthThreshold", "200")
    .getOrCreate()
)

alerts_df = spark.read.csv("csv_directory", header=True)
alerts_df1 = alerts_df.groupBy("value").count()

alerts_df1.show(5)


# Старт стримінгу та виведення результатів на екран
query = (
    alerts_df1.writeStream.trigger(availableNow=True)
    .outputMode("complete")
    # .outputMode("update")
    .format("console")
    .start()
)

# Чекаємо кінця роботи стримінгу
query.awaitTermination()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    TimestampType,
)

# Створення SparkSession
spark = SparkSession.builder.appName("CSVStream").getOrCreate()

# Визначення схеми для CSV-файлу
schema = StructType(
    [
        StructField("id", IntegerType(), nullable=True),
        StructField("event_time", TimestampType(), nullable=True),
        StructField("value", StringType(), nullable=True),
    ]
)

# Читання потокових даних із CSV-файлу
csvDF = (
    spark.readStream.option("sep", ",")
    .option("header", True)
    .option("maxFilesPerTrigger", 1)
    .schema(schema)
    .csv("csv_directory")
)



# Старт стримінгу й виведення результатів на екран
query = (
    csvDF.writeStream.trigger(availableNow=True)
    # .trigger(processingTime='10 seconds')
    .outputMode("append")
    .format("console")
    .start()
)

# Чекаємо кінця роботи стримінгу
query.awaitTermination()